In [1]:
import pandas as pd
from pathlib import Path

In [16]:
import re
import csv
import time
import wandb
import numpy as np
import torch
import os
import argparse
from transform import Apple_train_transform, Apple_test_transform
from dataset import AppleDataset
import torchvision.models as models
from model import My_regnet, My_efficientnet
from torch import nn, optim
import random
from utils import WarmupReduceLROnPlateau

## 通过权重合并

In [12]:
DATA_DIR = Path('/home/image003/xunfei/jicheng')
file_list = [str(i) for i in DATA_DIR.glob("*")]
file_list

['/home/image003/xunfei/jicheng/apple-regnet-mixup-0.9648418372343551_9617.pth',
 '/home/image003/xunfei/jicheng/apple-regnet-mixup-0.9653315052394477_9617.pth',
 '/home/image003/xunfei/jicheng/apple-regnet-mixup-0.9660170404465772_9617.pth',
 '/home/image003/xunfei/jicheng/apple-regnet-mixup-0.9666046420526883_9617.pth',
 '/home/image003/xunfei/jicheng/apple-regnet-mixup-0.9668984428557438_9617.pth',
 '/home/image003/xunfei/jicheng/apple-regnet-mixup-0.9683674468710215_9617.pth']

In [17]:
from  tqdm import tqdm

In [22]:
model = My_regnet(num_classes=9)
device = 'cuda:1'
file_list_2 = []
model = model.to(device)
Data_test = AppleDataset("/home/medicaldata/WTJData/xunfei/苹果病害图像识别挑战赛公开数据/", "test", transform=Apple_test_transform)
testloader = torch.utils.data.DataLoader(Data_test,
                                         batch_size=64,
                                         shuffle=False, num_workers=8)
print("ok")
for checkpoint_name in  tqdm(file_list):
    checkpoint = torch.load(checkpoint_name,map_location=device)
    
    accuracy = re.search(r'\d+\.\d+', checkpoint_name).group()

    model.load_state_dict(checkpoint['model_state_dict'])
    model.to(device)
    del checkpoint
    

    label_map = {0: "d1", 1: "d2",  2: "d3",  3: "d4",  4: "d5",  5: "d6",  6: "d7",  7: "d8",  8: "d9"}
    model.eval()
    with torch.no_grad(), open(f'./results/apple-results-{accuracy}.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['uuid', 'label'])  # Write the header row

        for i, (inputs, labels, img_name) in enumerate(testloader):
            inputs = inputs.to(device)
            output = model(inputs)
            output = output.argmax(1)

            for j in range(len(output)):
                writer.writerow([img_name[j], label_map[output[j].item()]])  # Write the UUID and label to the CSV file
            # print(output)
    file_list_2.append(f'./results/apple-results-{accuracy}.csv')
print("ok")

ok


  0%|          | 0/6 [00:00<?, ?it/s]

tensor([0, 8, 1, 8, 0, 2, 0, 7, 7, 7, 8, 2, 8, 8, 2, 8, 8, 8, 8, 7, 8, 8, 8, 7,
        2, 7, 6, 2, 8, 8, 2, 7, 8, 8, 2, 2, 8, 5, 7, 8, 2, 6, 2, 0, 7, 0, 2, 5,
        2, 8, 2, 8, 2, 2, 0, 2, 2, 8, 8, 7, 2, 0, 2, 8], device='cuda:1')
tensor([8, 8, 7, 6, 7, 7, 8, 2, 8, 2, 2, 7, 0, 2, 8, 6, 8, 2, 0, 6, 0, 7, 4, 2,
        6, 7, 8, 2, 2, 8, 6, 8, 2, 6, 7, 7, 2, 7, 8, 4, 6, 6, 8, 2, 8, 8, 7, 7,
        6, 8, 7, 3, 4, 6, 7, 2, 4, 7, 2, 7, 8, 7, 2, 0], device='cuda:1')
tensor([8, 8, 8, 8, 6, 8, 6, 8, 4, 8, 8, 7, 3, 4, 8, 8, 8, 2, 8, 8, 7, 8, 7, 7,
        8, 4, 8, 8, 8, 8, 4, 8, 8, 5, 8, 2, 2, 8, 0, 8, 7, 8, 8, 2, 8, 4, 8, 2,
        4, 2, 8, 6, 8, 6, 8, 8, 2, 8, 8, 4, 2, 2, 2, 2], device='cuda:1')
tensor([8, 7, 7, 8, 8, 2, 8, 8, 8, 6, 4, 7, 2, 8, 4, 7, 8, 8, 8, 2, 2, 2, 7, 8,
        2, 8, 7, 8, 7, 2, 6, 7, 8, 8, 2, 4, 8, 7, 7, 8, 7, 8, 2, 8, 7, 8, 8, 6,
        2, 6, 2, 7, 8, 7, 6, 3, 7, 7, 8, 3, 2, 8, 8, 2], device='cuda:1')
tensor([7, 6, 6, 6, 2, 2, 8, 7, 7, 6, 8, 6, 8, 8, 6, 1, 8, 7, 3,

 17%|█▋        | 1/6 [02:28<12:20, 148.07s/it]

tensor([2, 8, 2, 1, 2, 8, 7, 0, 5, 7, 0, 7, 7, 2, 8, 8, 8, 2, 3],
       device='cuda:1')
tensor([0, 8, 1, 8, 0, 2, 0, 7, 7, 7, 8, 2, 8, 8, 2, 8, 8, 8, 8, 7, 8, 8, 8, 7,
        2, 7, 6, 2, 8, 8, 2, 7, 8, 8, 2, 2, 8, 5, 7, 8, 2, 6, 2, 0, 7, 0, 2, 5,
        2, 8, 2, 8, 2, 2, 0, 2, 2, 8, 8, 7, 2, 0, 2, 8], device='cuda:1')
tensor([8, 8, 7, 6, 7, 7, 8, 2, 8, 2, 2, 7, 0, 2, 8, 6, 8, 2, 0, 6, 0, 7, 4, 2,
        6, 7, 8, 2, 2, 8, 6, 8, 2, 6, 7, 7, 2, 7, 8, 4, 6, 6, 8, 2, 8, 8, 7, 7,
        6, 8, 7, 0, 4, 6, 7, 2, 4, 7, 2, 7, 8, 7, 2, 0], device='cuda:1')
tensor([8, 8, 8, 8, 6, 8, 6, 8, 4, 8, 8, 7, 3, 4, 8, 8, 8, 2, 8, 8, 8, 8, 7, 7,
        8, 4, 8, 8, 8, 8, 4, 8, 8, 5, 8, 2, 2, 8, 0, 8, 2, 8, 8, 2, 8, 4, 8, 2,
        4, 2, 8, 6, 8, 6, 8, 8, 2, 8, 8, 4, 2, 2, 2, 2], device='cuda:1')
tensor([8, 7, 7, 8, 8, 2, 8, 8, 8, 6, 4, 7, 2, 8, 4, 7, 8, 8, 8, 2, 2, 2, 7, 8,
        2, 8, 7, 8, 7, 8, 6, 7, 8, 8, 2, 4, 8, 7, 7, 8, 7, 8, 2, 8, 7, 8, 8, 6,
        2, 6, 2, 7, 8, 7, 6, 3, 7, 7, 8, 3, 2, 8

...

tensor([2, 8, 2, 1, 2, 8, 7, 0, 5, 7, 0, 7, 7, 2, 8, 8, 8, 2, 3],
       device='cuda:1')
tensor([0, 8, 1, 8, 0, 2, 0, 7, 7, 7, 8, 2, 8, 8, 2, 8, 8, 8, 8, 7, 8, 8, 8, 7,
        2, 7, 6, 2, 8, 8, 2, 7, 8, 8, 2, 2, 8, 5, 7, 8, 8, 6, 2, 0, 7, 0, 2, 5,
        2, 8, 2, 8, 2, 2, 0, 2, 2, 8, 8, 7, 2, 0, 2, 8], device='cuda:1')
tensor([8, 8, 7, 6, 7, 7, 8, 2, 8, 2, 2, 7, 0, 2, 8, 6, 8, 2, 0, 6, 0, 7, 4, 2,
        6, 7, 8, 2, 2, 8, 6, 8, 2, 6, 7, 7, 2, 7, 8, 4, 6, 6, 8, 2, 8, 8, 7, 7,
        6, 8, 7, 0, 4, 6, 7, 2, 4, 7, 2, 7, 8, 7, 2, 0], device='cuda:1')
tensor([8, 8, 8, 8, 6, 8, 6, 8, 4, 8, 8, 7, 3, 4, 8, 8, 8, 2, 8, 8, 7, 8, 7, 7,
        8, 4, 8, 8, 8, 8, 4, 8, 8, 5, 8, 2, 2, 8, 0, 8, 2, 8, 8, 2, 8, 4, 8, 2,
        4, 2, 8, 6, 8, 6, 8, 8, 2, 8, 8, 4, 2, 2, 2, 2], device='cuda:1')
tensor([8, 7, 7, 8, 8, 2, 8, 8, 8, 6, 4, 7, 2, 8, 4, 7, 8, 8, 8, 2, 2, 2, 7, 8,
        2, 8, 7, 8, 7, 2, 6, 7, 8, 8, 2, 4, 8, 7, 7, 8, 7, 6, 2, 8, 7, 8, 8, 6,
        2, 6, 2, 7, 8, 7, 6, 3, 7, 7, 8, 3, 2, 8

 83%|████████▎ | 5/6 [12:19<02:27, 147.91s/it]

tensor([2, 8, 2, 1, 2, 8, 7, 0, 5, 7, 0, 7, 7, 2, 8, 8, 8, 2, 3],
       device='cuda:1')
tensor([0, 8, 1, 8, 0, 2, 0, 7, 7, 7, 8, 2, 8, 8, 2, 8, 8, 8, 8, 7, 8, 8, 8, 7,
        2, 7, 6, 2, 8, 8, 2, 7, 8, 8, 2, 2, 8, 5, 7, 8, 2, 6, 2, 0, 7, 0, 2, 5,
        2, 8, 2, 8, 2, 2, 0, 2, 2, 8, 8, 7, 2, 0, 2, 8], device='cuda:1')
tensor([8, 8, 7, 6, 7, 7, 8, 2, 8, 2, 2, 7, 0, 2, 8, 6, 8, 2, 0, 6, 0, 7, 4, 2,
        6, 7, 8, 2, 2, 8, 6, 8, 2, 6, 7, 7, 2, 7, 8, 4, 6, 6, 8, 2, 8, 8, 7, 7,
        6, 8, 7, 3, 4, 6, 7, 2, 4, 7, 2, 7, 8, 7, 2, 0], device='cuda:1')
tensor([8, 8, 8, 8, 6, 8, 6, 8, 4, 8, 8, 7, 3, 4, 8, 8, 8, 2, 8, 8, 7, 8, 7, 7,
        8, 4, 8, 8, 8, 8, 4, 8, 8, 5, 8, 2, 2, 8, 0, 8, 2, 8, 8, 2, 8, 4, 8, 2,
        4, 2, 8, 6, 8, 6, 8, 8, 2, 8, 8, 4, 2, 2, 2, 2], device='cuda:1')
tensor([8, 7, 7, 8, 8, 2, 8, 8, 8, 6, 4, 7, 2, 8, 4, 7, 8, 8, 8, 2, 2, 2, 7, 8,
        2, 8, 7, 8, 7, 8, 6, 7, 8, 8, 2, 4, 8, 7, 7, 8, 7, 6, 2, 8, 7, 8, 8, 6,
        2, 6, 2, 7, 8, 7, 6, 3, 7, 7, 8, 3, 2, 8

100%|██████████| 6/6 [14:47<00:00, 147.92s/it]

tensor([2, 8, 2, 1, 2, 8, 7, 0, 5, 7, 0, 7, 7, 2, 8, 8, 8, 2, 3],
       device='cuda:1')
ok


In [23]:
pd_list = [ pd.read_csv(path) for path in file_list_2]

## 通过csv合并

In [27]:
# 合并表格
merged_df = pd.concat(pd_list,axis = 1)
# 创建ans表格
ans = pd.DataFrame()
ans["uuid"] = pd_list[0]["uuid"]
ans["label"] = merged_df['label'].mode(axis=1)[0]
ans 

,uuid,label
0,0001348010.jpg,d1
1,000623946.jpg,d9
2,000767853.jpg,d2
3,001043027.jpg,d9
4,001121349.jpg,d1
...,...,...
4366,999192160.jpg,d9
4367,999614360.jpg,d9
4368,999727804.jpg,d9
4369,9997443683.jpg,d3


In [28]:
ans.to_csv("./ans.csv")